In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
def prep_data_cpc():
    #read file
    csv_path="./cpc.csv"
    df = pd.read_csv(csv_path)

    #fill missing values
    doy = df.pop('doy')
    df['doys'] = np.sin((doy * 2 * np.pi)/365.24)
    df['doyc'] = np.cos((doy * 2 * np.pi)/365.24)
    sm = df.pop('sm')
    sm = sm / 1000.0
    df['sm'] = sm
    lai = df.pop('lai')
    lai = lai / 4
    df['lai'] = lai
    ir = df.pop('ir')
    ir_mean = ir.mean()
    ir_std = ir.std()
    ir = (ir - ir_mean) / ir_std
    df['ir'] = ir
    
    return df

def prep_data_uscrn():
    #read file
    csv_path="./uscrn.csv"
    df = pd.read_csv(csv_path)
    
    #fill missing values
    for i in range(1,6):
        sm = df['sm'+str(i)]
        bad_sm1 = (sm < 0)
        bad_sm2 = (sm > 1)
        sm[bad_sm1] = 0
        sm[bad_sm2] = 0
    df['sm']=df[['sm1','sm2','sm3','sm4','sm5']].apply(np.max,axis=1)
    for i in range(1,6):
        df.pop('sm'+str(i))
    df = df[df.sm != 0]
    df.reset_index(drop=True, inplace=True)
    
    doy = df.pop('doy')
    df['doys'] = np.sin((doy * 2 * np.pi)/365.24)
    df['doyc'] = np.cos((doy * 2 * np.pi)/365.24)
    
    return df

In [ ]:
df = prep_data_cpc()

In [ ]:
df.head()

In [ ]:
df.describe(include='all').transpose()

In [ ]:
#plot data
s = 0
e = s + 1460
plot_features = df[:][s:e]
plot_features.index = range(0,e-s)
_ = plot_features.plot(subplots=True)

In [ ]:
def plot_fft(plot_col='sm', ylim=None):
    fft = tf.signal.rfft(df[plot_col])
    f_per_dataset = np.arange(0, len(fft))

    n_samples_d = len(df[plot_col])
    days_per_year = 365.2524
    years_per_dataset = n_samples_d/(days_per_year)

    f_per_year = f_per_dataset/years_per_dataset
    plt.figure()
    plt.step(f_per_year, np.abs(fft))
    plt.xscale('log')
    plt.title(plot_col)
    if ylim is not None:
        plt.ylim(0, ylim)
    plt.xlim([0.1, max(plt.xlim())])
    plt.xticks([0.25, 1], labels=['1/4-Years', '1/Year'])
    _ = plt.xlabel('Frequency (log scale)')

plot_fft(plot_col='sm', ylim=8000)
plot_fft(plot_col='ir')

In [ ]:
#set prediction point (e.g. tomorrow = 1 day shift)
PRED_SHIFT=2
#how much to shift the input
INPUT_SHIFT=(0 if PRED_SHIFT > 0 else 1)

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]
num_features = df.shape[1]
sm_index = column_indices['sm']
print(column_indices)

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
w2 = WindowGenerator(input_width=7, label_width=1, shift=PRED_SHIFT,
                     label_columns=['sm'])
w2

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])
    
  #set "sm" to 0 for the last-time step when SHIFT=0
  if PRED_SHIFT == 0:
      mask = np.ones(shape=inputs.shape.as_list()[1:])
      mask[self.input_width - 1, sm_index] = 0
      inputs = tf.multiply(inputs, mask)

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
# Stack three slices, the length of the total window.
shift = 100000
example_window = tf.stack([np.array(train_df[shift:shift+w2.total_window_size]),
                           np.array(train_df[shift+100:shift+100+w2.total_window_size]),
                           np.array(train_df[shift+200:shift+200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
w2.example = example_inputs, example_labels

In [ ]:
def plot(self, model=None, plot_col='sm', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    if plot_col == 'sm':
        plt.ylabel(f'{plot_col} m/m')
        if PRED_SHIFT == 0:
            end = -1
        else:
            end = None
        plt.plot(self.input_indices[:end], inputs[n, :end, plot_col_index],
             label='Inputs', marker='.', zorder=-10)
    else:
        plt.ylabel(f'{plot_col}')
        plt.plot(self.input_indices[:], inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Days')

WindowGenerator.plot = plot

In [ ]:
w2.plot()

In [ ]:
w2.plot(plot_col='ir')

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
# Each element is an (inputs, label) pair.
w2.train.element_spec

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
single_step_window = WindowGenerator(
    input_width=1+INPUT_SHIFT, label_width=1, shift=PRED_SHIFT,
    label_columns=['sm'])
single_step_window

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=patience,
                                                mode='min')
  

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
val_performance = {}
performance = {}
np.random.seed(7)

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    paddings = tf.constant([[0,0],[INPUT_SHIFT,0]])
    res = tf.pad(res, paddings, "SYMMETRIC")
    res = tf.roll(res, shift=INPUT_SHIFT, axis=1)
    
    return res[:, INPUT_SHIFT:, tf.newaxis] + delta

In [ ]:
baseline = ResidualWrapper(
    tf.keras.layers.Lambda(lambda x: 0.0)
)

In [ ]:
history = compile_and_fit(baseline, single_step_window)

val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

In [ ]:
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=PRED_SHIFT,
    label_columns=['sm'])

wide_window

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)

In [ ]:
wide_window.plot(baseline)

In [ ]:
wide_window.plot(baseline, plot_col='lai')

In [ ]:
linear = ResidualWrapper(
    tf.keras.Sequential([
        tf.keras.layers.Dense(units=1, kernel_initializer=tf.initializers.zeros(),
                              kernel_regularizer=tf.keras.regularizers.l2(1e-4))
    ])
)

In [ ]:
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', linear(single_step_window.example[0]).shape)

In [ ]:
history = compile_and_fit(linear, single_step_window)

val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)

In [ ]:
wide_window.plot(linear)

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=linear.model.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

In [ ]:
dense =  ResidualWrapper(
    tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=1, kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                              kernel_initializer=tf.initializers.zeros())
    ])
)
history = compile_and_fit(dense, single_step_window)

val_performance['Dense'] = dense.evaluate(single_step_window.val)
performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)

In [ ]:
wide_window.plot(dense)

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    paddings = tf.constant([[0,0],[1+INPUT_SHIFT,0]])
    res = tf.pad(res, paddings, "SYMMETRIC")
    res1 = tf.roll(res, shift=INPUT_SHIFT, axis=1)
    res2 = tf.roll(res, shift=1+INPUT_SHIFT, axis=1)
    res = 2 * res1 - res2
    res = res[:, 1+INPUT_SHIFT:, tf.newaxis]
    return  res + delta

In [ ]:
baseline_multi = ResidualWrapper(
    tf.keras.layers.Lambda(lambda x: 0.0)
)

In [ ]:
history = compile_and_fit(baseline_multi, wide_window)

IPython.display.clear_output()
val_performance['Baseline multi'] = baseline_multi.evaluate(wide_window.val)
performance['Baseline multi'] = baseline_multi.evaluate(wide_window.test, verbose=0)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline_multi(wide_window.example[0]).shape)

In [ ]:
wide_window.plot(baseline_multi)

In [ ]:
# MULTI-STEP model
CONV_WIDTH = 3
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=PRED_SHIFT,
    label_columns=['sm'])

conv_window

In [ ]:
conv_window.plot()
plt.title(f"Given {CONV_WIDTH} days of inputs, predict 1 day into the future.")

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    paddings = tf.constant([[0,0],[1+INPUT_SHIFT,0]])
    res = tf.pad(res, paddings, "SYMMETRIC")
    res1 = tf.roll(res, shift=INPUT_SHIFT, axis=1)
    res2 = tf.roll(res, shift=1+INPUT_SHIFT, axis=1)
    res = 2 * res1 - res2
    res = res[:, -1:, tf.newaxis]
    return  res + delta

In [ ]:
multi_step_dense = ResidualWrapper(
    tf.keras.Sequential([
        # Shape: (time, features) => (time*features)
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=1, kernel_initializer=tf.initializers.zeros(), 
                              kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        # Add back the time dimension.
        # Shape: (outputs) => (1, outputs)
        tf.keras.layers.Reshape([1, -1]),
    ])
)

In [ ]:
print('Input shape:', conv_window.example[0].shape)
print('Output shape:', multi_step_dense(conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(multi_step_dense, conv_window)

IPython.display.clear_output()
val_performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.val)
performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.test, verbose=0)

In [ ]:
conv_window.plot(multi_step_dense)

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    res1 = tf.roll(res, shift=INPUT_SHIFT, axis=1)
    res2 = tf.roll(res, shift=1+INPUT_SHIFT, axis=1)
    res = 2 * res1 - res2
    res = res[:, (CONV_WIDTH - 1):, tf.newaxis]
    return res + delta

In [ ]:
conv_model = ResidualWrapper(
    tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32,
                               kernel_size=(CONV_WIDTH,),
                               activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=32, activation='relu', 
                              kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=1, kernel_initializer=tf.initializers.zeros(),
                              kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    ])
)

In [ ]:
print("Conv window")
print('Input shape:', conv_window.example[0].shape)
print('Labels shape:', conv_window.example[1].shape)
print('Output shape:', conv_model(conv_window.example[0]).shape)

In [ ]:
print("Wide window")
print('Input shape:', wide_window.example[0].shape)
print('Labels shape:', wide_window.example[1].shape)
print('Output shape:', conv_model(wide_window.example[0]).shape)

In [ ]:
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=PRED_SHIFT,
    label_columns=['sm'])

wide_conv_window

In [ ]:
print("Wide conv window")
print('Input shape:', wide_conv_window.example[0].shape)
print('Labels shape:', wide_conv_window.example[1].shape)
print('Output shape:', conv_model(wide_conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(conv_model, wide_conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(wide_conv_window.val)
performance['Conv'] = conv_model.evaluate(wide_conv_window.test, verbose=0)

In [ ]:
conv_window.plot(conv_model)

In [ ]:
wide_conv_window.plot(conv_model)

In [ ]:
wide_conv_window.plot(conv_model, plot_col='gf')

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    paddings = tf.constant([[0,0],[1+INPUT_SHIFT,0]])
    res = tf.pad(res, paddings, "SYMMETRIC")
    res1 = tf.roll(res, shift=INPUT_SHIFT, axis=1)
    res2 = tf.roll(res, shift=1+INPUT_SHIFT, axis=1)
    res = 2 * res1 - res2
    res = res[:, 1+INPUT_SHIFT:, tf.newaxis]
    return  res + delta

In [ ]:
lstm_model = ResidualWrapper(
    tf.keras.models.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=True,
                             kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(units=1, kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                              kernel_initializer=tf.initializers.zeros())
    ])
)

In [ ]:
print("Wide window")
print('Input shape:', wide_window.example[0].shape)
print('Labels shape:', wide_window.example[1].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

In [ ]:
x = np.arange(len(performance))
width = 0.3
metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.ylabel('mean_absolute_error [sm]')
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
_ = plt.legend()

In [ ]:
for name, value in performance.items():
  print(f'{name:12s}: {value[1]:0.4f}')

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=['sm'])

In [ ]:
multi_window.plot()
multi_window

In [ ]:
multi_val_performance = {}
multi_performance = {}

In [ ]:
class ResidualWrapperLast(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, -1:, sm_index]
    res = res[:, :, tf.newaxis]
    res = tf.tile(res, [1, OUT_STEPS, 1])
    return  res + delta

class ResidualWrapperRepeat(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    res = inputs[:, :, sm_index]
    res = res[:, :, tf.newaxis]
    return  res + delta

In [ ]:
baseline_multi_output_last = ResidualWrapperLast(
    tf.keras.layers.Lambda(lambda x: 0.0)
)

In [ ]:
print("Multi window")
print('Input shape:', multi_window.example[0].shape)
print('Labels shape:', multi_window.example[1].shape)
print('Output shape:', baseline_multi_output_last(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(baseline_multi_output_last, wide_window)

IPython.display.clear_output()
multi_val_performance['Last'] = baseline_multi_output_last.evaluate(multi_window.val)
multi_performance['Last'] = baseline_multi_output_last.evaluate(multi_window.test, verbose=0)


In [ ]:
multi_window.plot(baseline_multi_output_last)

In [ ]:
baseline_multi_output_repeat = ResidualWrapperRepeat(
    tf.keras.layers.Lambda(lambda x: 0.0)
)

In [ ]:
history = compile_and_fit(baseline_multi_output_repeat, wide_window)

IPython.display.clear_output()
multi_val_performance['Repeat'] = baseline_multi_output_repeat.evaluate(multi_window.val)
multi_performance['Repeat'] = baseline_multi_output_repeat.evaluate(multi_window.test, verbose=0)

In [ ]:
multi_window.plot(baseline_multi_output_repeat)

In [ ]:
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, 1])
])


In [ ]:
history = compile_and_fit(multi_linear_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)


In [ ]:
multi_window.plot(multi_linear_model)

In [ ]:
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, 1])
])

In [ ]:
history = compile_and_fit(multi_dense_model, multi_window)

IPython.display.clear_output()
multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=0)

In [ ]:
multi_window.plot(multi_dense_model)

In [ ]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, 1])
])


In [ ]:
history = compile_and_fit(multi_conv_model, multi_window)

IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)


In [ ]:
multi_window.plot(multi_conv_model)

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, 1])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)

In [ ]:
multi_window.plot(multi_lstm_model)

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

In [ ]:
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the LSTM state.
  prediction, state = self.warmup(inputs)

  # Insert the first prediction.
  predictions.append(prediction)

  # Run the rest of the prediction steps.
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output.
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

In [ ]:
multi_window.plot(feedback_model)

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()

In [ ]:
for name, value in multi_performance.items():
  print(f'{name:8s}: {value[1]:0.4f}')